In [2]:
#импортируем нужные библиотеки

import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import datetime
import _datetime
import numpy as np
import math
import xlrd
import numpy

In [236]:
#параметры, задаваемые пользователем
perv_mesyac=3 #первый полный месяц продажи
str_zapas=0 #страховой запас
mesyac_tz=3 #количество месяцев ТЗ
malenk_prodazhi=10 #количество проданных штук, которые считаются маленькими продажами
kolvo_magazov=3#количество магазинов в категории

In [8]:
#коэффициент сезонности

rb=xlrd.open_workbook('/Users/anna/Downloads/оборот розницы.xlsx') 
sheet=rb.sheet_by_index(0) 
oborot_rozn=[] 
for rownum in range(sheet.nrows): 
    row=sheet.row_values(rownum) 
    for c_el in row: 
        oborot_rozn.append(c_el)
        
        
data_month=[]
data_year=[]
obor=[]

for i in range(0,len(oborot_rozn),3):
    data_month.append(oborot_rozn[i])
for i in range(1,len(oborot_rozn),3):
    data_year.append(oborot_rozn[i])
for i in range(2,len(oborot_rozn),3):
    obor.append(oborot_rozn[i])

x=range(1,37)
otkl_x=[]
otkl_x_kv=[]
otkl_obor=[]
umn=[]
for i in range(36):
    
    otkl_x_zn=x[i]-(sum(x)/len(x))
    otkl_x.append(otkl_x_zn)
    otkl_x_kv_zn=otkl_x_zn*otkl_x_zn
    otkl_x_kv.append(otkl_x_kv_zn)
    otkl_obor_zn=obor[i+1]-(sum(obor[1:37])/len(obor[1:37]))
    otkl_obor.append(otkl_obor_zn)
    umn_zn=otkl_x_zn*otkl_obor_zn
    umn.append(umn_zn)

b=sum(umn)/sum(otkl_x_kv)
a=sum(obor[1:37])/len(obor[1:37])-(sum(x)/len(x))*b
trend=[]
for i in range(36):
    trend.append(a+b*x[i])

otkl_fact_trend=[]
for i in range(36):
    otkl_fact_trend.append(obor[1+i]/trend[i])
    
sr_otl_month=[]
for i in range(12):
    sr_otl_month.append((otkl_fact_trend[i]+otkl_fact_trend[i+12]+otkl_fact_trend[i+24])/3)

obs_index_sez=sum(sr_otl_month)/len(sr_otl_month)
koef_sez=[]
for i in range(12):
    koef_sez.append(sr_otl_month[i]/obs_index_sez)      

In [15]:
m=int(13-data_month[1])
koef_final_sez=koef_sez[m:]+koef_sez[:m]

In [119]:
koef_final=koef_final_sez[(perv_mesyac-1):]+koef_final_sez[:(perv_mesyac-1)]

In [86]:
koef_final_final=koef_final_sez+koef_final_sez+koef_final_sez

In [53]:
rb=xlrd.open_workbook('/Users/anna/Downloads/первый год.xlsx') 
sheet=rb.sheet_by_index(0) 
first_year=[] 
for rownum in range(sheet.nrows): 
    row=sheet.row_values(rownum) 
    for c_el in row: 
        first_year.append(c_el)

In [88]:
first_year_kor=[]
for i in range(int(len(first_year)/13)):
    dl=0
    for u in range(12):
        dl+=1
        first_year_kor.append(first_year[u+13*i]/koef_final_final[int(first_year[13*(i+1)-1])-2+dl])

In [29]:
kolvo_magaz=int(len(first_year)/13)

In [101]:
m_obs=[]
for i in range(12):
    ob=0
    for g in range(kolvo_magaz):
        
        ob+=first_year_kor[i+g*12]
    m_obs.append(ob/kolvo_magaz)        

1063030.6774235484
313032.5430302027
871209.2094176573
350304.7186504633
683731.4228692189
449162.29096888646
833445.1423404702
337941.19897197385
498392.127238397
482262.3087192915
693414.6405042488
341257.0075330868
611832.939198063
374550.6786921282
991968.2121009484
390939.3865790651
785004.518499844
431140.2851867787
809214.1807286796
944981.0119097991
832987.1709735473
682997.1992071529
943125.5422274488
143046.02055312824


In [111]:
sr_oborot_mesyac=oborot_posl_god/12 
otl_perv_ot_posl=(sum(m_obs)/len(m_obs))/sr_oborot_mesyac

In [103]:
m_obs_mean=sum(m_obs)/len(m_obs)

In [105]:
otkl_month=[]
for i in range(12):
    otkl_month.append(m_obs[i]/m_obs_mean)

In [120]:
progn=[]
for g in range(12):
    progn.append(koef_final[g]*otkl_month[g]*otl_perv_ot_posl*sr_oborot_mesyac)

In [122]:
sum_prog=0
for i in range(mesyac_tz):
    sum_prog+=progn[i]

In [125]:
sum_prog_strax=sum_prog*(1+str_zapas)

In [ ]:
oborot_posl_god=7043776.965  #средний оборот по магазинам категории нового магазина

In [139]:
rb=xlrd.open_workbook('/Users/anna/Downloads/продажи_УТЗ.xlsx') 
sheet=rb.sheet_by_index(0) 
a=[] 
for rownum in range(sheet.nrows): 
    row=sheet.row_values(rownum) 
    for c_el in row: 
        a.append(c_el)

In [140]:
artic=[]
summa_prodaz=[]
kolvo_prodaz=[]


for i in range(0,len(a),5):
    artic.append(a[i])
for i in range(3,len(a),5):
    summa_prodaz.append(a[i])
for i in range(4,len(a),5):
    kolvo_prodaz.append(a[i])


In [146]:
rb=xlrd.open_workbook('/Users/anna/Downloads/розничные цены.xlsx') 
sheet=rb.sheet_by_index(0) 
a=[] 
for rownum in range(sheet.nrows): 
    row=sheet.row_values(rownum) 
    for c_el in row: 
        a.append(c_el)

In [147]:
artic_rozn_cena=[]
rozn_cena=[]

for i in range(0,len(a),9):
    artic_rozn_cena.append(a[i])
for i in range(8,len(a),9):
    rozn_cena.append(a[i])


In [220]:
rb=xlrd.open_workbook('/Users/anna/Downloads/матрица розницы.xlsx') 
sheet=rb.sheet_by_index(0) 
a=[] 
for rownum in range(sheet.nrows): 
    row=sheet.row_values(rownum) 
    for c_el in row: 
        a.append(c_el)

In [221]:
artic_rozn=[]
new_rozn=[]
vitrina_rozn=[]
korobka_rozn=[]
kolvo_magaz_s_pr=[]

for i in range(0,len(a),6):
    artic_rozn.append(a[i])
for i in range(2,len(a),6):
    new_rozn.append(a[i])
for i in range(3,len(a),6):
    vitrina_rozn.append(a[i])
for i in range(4,len(a),6):
    korobka_rozn.append(a[i])
for i in range(4,len(a),6):
    kolvo_magaz_s_pr.append(3)

In [222]:
obor_rozn=[]
kolv_rozn=[]
stoim_rozn=[]
for i in range(len(artic_rozn)):
    r=0
    p=0
    stoim=0
    for h in range(len(artic)):
        if artic_rozn[i]==artic[h]:
            r=summa_prodaz[h]
            p=kolvo_prodaz[h]
            try:
                stoim=int(r)/int(p)
            except:
                stoim=0
    if stoim==0:
        for y in range(len(rozn_cena)):
            if artic_rozn[i]==artic_rozn_cena[y]:
                stoim=rozn_cena[y]
    
        
    obor_rozn.append(r)
    kolv_rozn.append(p)
    stoim_rozn.append(stoim)

In [223]:
df=pd.DataFrame(list(zip(artic_rozn[1:],vitrina_rozn[1:],korobka_rozn[1:],obor_rozn[1:],kolv_rozn[1:],stoim_rozn[1:])),columns=['Артикул','Витрина','Коробка','Оборот категории','Количество проданных шт','Цена 1 шт'])

In [224]:
df2=df.sort_values(by=['Оборот категории'], ascending=False)

In [231]:
df2['Количество магазинов с продажами']=3

In [225]:

summ_za_god=sum(df2['Оборот категории']) #оборот категории за последний год

In [226]:
#скорректированная сумма
df2['Скорректированная сумма']=1
for i in range(len(df2)):
    try:
        df2['Скорректированная сумма'][i]=df2['Оборот категории'][i]*sum_prog_strax/summ_za_god
    except:
        0

/Users/anna/opt/anaconda3/envs/denis/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [241]:
#рассчетные продажи на новый магазин, исходя из заданного количества месяцев ТЗ
df2['Расчетные продажи']=1
for i in range(len(df2)):
    try:
        if df2['Количество магазинов с продажами'][i]<(kolvo_magazov/2) and df2['Количество проданных шт']<malenk_prodazhi:
            df2['Расчетные продажи'][i]=0
        else:
            if df2['Скорректированная сумма'][i]/df2['Цена 1 шт'][i]<1:
                df2['Расчетные продажи'][i]=1
            else:
                df2['Расчетные продажи'][i]=round(df2['Скорректированная сумма'][i]/df2['Цена 1 шт'][i])
    except:
        df2['Расчетные продажи'][i]=0
    

/Users/anna/opt/anaconda3/envs/denis/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/anna/opt/anaconda3/envs/denis/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [228]:
df2['Стоимость итого']=df2['Расчетные продажи']*df2['Цена 1 шт']

In [229]:
df2['Стоимость витрины']=df2['Витрина']*df2['Цена 1 шт']

In [243]:
df2['Сколько нужно взять без витрины']=df2['Расчетные продажи']-df2['Витрина']

In [256]:
for i in range(len(df2)):
    if df2['Сколько нужно взять без витрины'][i]<0:
        df2['Сколько нужно взять без витрины'][i]=0
    

/Users/anna/opt/anaconda3/envs/denis/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [265]:
df2.to_excel('hu.xlsx')

In [258]:
t10=0
for i in range(len(df2)):
    if df2['Сколько нужно взять без витрины'][i]>0:
        t10+=df2['Стоимость итого'][i]

In [261]:
otklonen=t10/sum_prog_strax

In [263]:
t10

789810.3205829326

In [262]:
otklonen

0.43040130429470985

In [ ]:
dobor1=[]
dobor_sum=[]
for i in range(len(df2)):
    try:
        if vitrina_rozn[i]>=ras_prod[i]:
            q=0
        else:
            q=math.ceil((ras_prod[i]-vitrina_rozn[i])/tt)

        if zakaz_fin[i]<q:
            fpo=zakaz_fin[i]
        else:
            if vitrina_rozn[i]>=ras_prod[i]:
                ll=0
            else:
                ll=math.ceil((ras_prod[i]-vitrina_rozn[i])/tt)

        dobor_sum.append(ll*stoim_rozn[i])
        dobor1.apend(ll)        
    except: 
        dobor1.append(0)
        dobor_sum.append(0)

In [268]:
dobor1=[]
dobor_sum=[]
for i in range(len(artic_rozn)):
    try:
        if vitrina_rozn[i]>=ras_prod[i]:
            q=0
        else:
            q=math.ceil((ras_prod[i]-vitrina_rozn[i])/tt)

        if zakaz_fin[i]<q:
            fpo=zakaz_fin[i]
        else:
            if vitrina_rozn[i]>=ras_prod[i]:
                ll=0
            else:
                ll=math.ceil((ras_prod[i]-vitrina_rozn[i])/tt)

        dobor_sum.append(ll*stoim_rozn[i])
        dobor1.apend(ll)        
    except: 
        dobor1.append(0)
        dobor_sum.append(0)

In [273]:
itog=[]
itog_kol=[]
for i in range(len(artic_rozn)):
    try:
        it_kol=dobor1[i]+vitrina_rozn[i]
        it=(dobor1[i]+vitrina_rozn[i])*stoim_rozn[i]
        itog.append(it)
        itog_kol.append(it_kol)
    except:
        itog.append(0)
        itog_kol.append(0)

In [285]:
dobor2=[]
for i in range(len(artic_rozn)):
    ret=0
    if ((sum(itog)+ret+stoim_rozn[i])<(sum_prog_strax+sum(sum_vitr[1:]))) and (itog_kol[i]!=ras_prod[i]) and (itog_kol[i]!=vitrina_rozn[i]):
        znach=1
        ret+=stoim_rozn[i]
    else:
        znach=0
    dobor2.append(znach)

In [290]:
fin_st_fin=[]
fin_fin=[]
for i in range(len(artic_rozn)):
    try:
        poi=(dobor2[i]+itog_kol[i])/korobka_rozn[i]
        if poi!=poi//1 and (dobor2[i]+itog_kol[i])!=0:
            fin=math.ceil(poi)*korobka_rozn[i]
            fin_st=fin*stoim_rozn[i]
        else:
            fin=dobor2[i]+itog_kol[i]
            fin_st=fin*stoim_rozn[i]
    except:
        fin=0
        fin_st=0
    fin_fin.append(fin)
    fin_st_fin.append(fin_st)

In [304]:
status=[]
for i in range(len(artic_rozn)):
    try:
        if fin_fin[i]>kolvo_infor[i] and kolvo_infor[i]>0:
            stat='Частичное наличие на ИНФОР'
        else:
            if kolvo_infor[i]<=0:
                stat='Нет на ИНФОРе'
            else:
                if fin_fin[i]<kolvo_infor[i]:
                    stat='Полное наличие на ИНФОР'
    except:
        stat=''
    status.append(stat)

In [306]:
#формирование итоговой таблицы
table=pd.DataFrame(list(zip(artic_rozn,fin_fin,fin_st_fin,status)))

In [307]:
table

,0,1,2,3
0,Артикул,0.0,0.000000,
1,ACK-384027,1.0,0.000000,Полное наличие на ИНФОР
2,ALT-5002F-RU,1.0,6956.666667,Полное наличие на ИНФОР
3,A8024,1.0,0.000000,Полное наличие на ИНФОР
4,PAW-04048-34,1.0,2461.428571,Полное наличие на ИНФОР
5,ALH-11102-P,1.0,395.857143,Полное наличие на ИНФОР
6,N3010,1.0,105.250000,Полное наличие на ИНФОР
7,AWT-ERSK03,1.0,5590.000000,Нет на ИНФОРе
8,AEB-H403,1.0,280.000000,Нет на ИНФОРе
9,AWT-ERS21,2.0,965.000000,Нет на ИНФОРе
